In [28]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()


from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from model_set import TakahashiNet

def train(x_train, y_train,x_val, y_val, x_test, y_test, batchsize = 8, epochs = 12):
        if tf.test.is_gpu_available():
                acc = []
                f1 = []
                prec = []
                recall = []
                base = 0.0
                
                for i in range(10):
                        # model = EEGNet(y.shape[1], Chans = x.shape[1], Samples = x.shape[2], 
                        #                 dropoutRate = 0.25, kernLength = 512, F1 = 32, 
                        #                 D = 8, F2 = 64, norm_rate = 0.25, dropoutType = 'Dropout')
                        model = TakahashiNet(output_classes = y_train.shape[1], Chans = x_train.shape[1], Samples = x_train.shape[2])
                        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
                        checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=False,save_best_only=True)
                        print(f"Training on fold {i + 1}/10")
                        hist = model.fit(x_train, y_train,
                                batch_size=batchsize, 
                                epochs = epochs, 
                                verbose = False, 
                                validation_data = (x_val,y_val),
                                callbacks = [checkpointer])
                        y_preds = model.predict(x_test, verbose = False)
                        predicted = np.argmax(y_preds, axis=1)
                        ground_truth = np.argmax(y_test, axis=1)
                        
                        acc.append(accuracy_score(ground_truth, predicted))
                        f1.append(f1_score(ground_truth, predicted, average = 'macro'))
                        prec.append(precision_score(ground_truth, predicted, average = 'macro'))
                        recall.append(recall_score(ground_truth, predicted, average = 'macro'))
                        tf.keras.backend.clear_session()
                
                return hist, np.array(acc), np.array(prec), np.array(recall), np.array(f1)
        else:
                print('NO GPU!')
                print(tf.config.list_physical_devices('GPU'))

def print_eval(acc, prec, recall, f1):
    for i in range(len(acc)):
        print(f"{acc[i]:.5f}, {prec[i]:.5f}, {recall[i]:.5f}, {f1[i]:.5f}")
    print(f"{acc.mean():.5f}, {prec.mean():.5f}, {recall.mean():.5f}, {f1.mean():.5f}")
    print(f"{acc.std():.5f}, {prec.std():.5f}, {recall.std():.5f}, {f1.std():.5f}")


In [4]:
import scipy.io
from sklearn.model_selection import train_test_split, StratifiedKFold
from capilab_dataset2 import load_target, butterworth_bpf
import numpy as np

In [36]:
# Takahashi --------------------------------------------------------------------------------------------

fs = 500
duration = 2
sample = fs * duration
ch = 19
hp = 0.4
lp = 30

data, label = load_target('Takahashi_JulyData')
data = butterworth_bpf(data, hp, lp, fs)


#select best channels from GA 1011001101101100110
# best_by_ga = [0, 1, 2, 3, 5, 8, 15, 16, 17, 18]

best_by_ga = [0, 2, 3, 6, 7, 9, 10, 12, 13, 16, 17]
data = data[:,best_by_ga,:]

d,x_test,  l, y_test = train_test_split(data, label, test_size = .15, stratify = label, random_state = 420)
x_train,x_val,  y_train, y_val = train_test_split(d, l, test_size = .10, stratify = l, random_state = 420)

tkh_hist, tkh_acc, tkh_prec, tkh_recall, tkh_f1 = train(x_train, y_train,x_val, y_val, x_test, y_test, 10)
print_eval(tkh_acc, tkh_prec, tkh_recall, tkh_f1)

Training on fold 1/10
Training on fold 2/10
Training on fold 3/10
Training on fold 4/10
Training on fold 5/10
Training on fold 6/10
Training on fold 7/10
Training on fold 8/10
Training on fold 9/10
Training on fold 10/10
0.56667, 0.64090, 0.56719, 0.57983
0.43333, 0.56809, 0.43626, 0.37808
0.65556, 0.66381, 0.65810, 0.65179
0.43333, 0.52766, 0.43775, 0.39064
0.62222, 0.64673, 0.62648, 0.59682
0.52222, 0.53298, 0.52717, 0.50196
0.50000, 0.57612, 0.50000, 0.51620
0.60000, 0.59535, 0.59881, 0.58852
0.50000, 0.57502, 0.50247, 0.51755
0.52222, 0.47808, 0.52569, 0.48437
0.53556, 0.58047, 0.53799, 0.52058
0.07128, 0.05562, 0.07084, 0.08369


In [35]:
# Sugiyama --------------------------------------------------------------------------------------------
fs = 500
duration = 2
sample = fs * duration
ch = 19
hp = 0.4
lp = 30

data, label = load_target('Sugiyama_JulyData')

data = butterworth_bpf(data, hp, lp, fs)
best_by_ga = [0, 2, 3, 6, 7, 9, 10, 12, 13, 16, 17]

data = data[:,best_by_ga,:]
d,x_test,  l, y_test = train_test_split(data, label, test_size = .15, stratify = label, random_state = 420)
x_train,x_val,  y_train, y_val = train_test_split(d, l, test_size = .10, stratify = l, random_state = 420)

sgym_hist, sgym_acc, sgym_prec, sgym_recall, sgym_f1 = train(x_train, y_train,x_val, y_val, x_test, y_test, 10)
print_eval(sgym_acc, sgym_prec, sgym_recall, sgym_f1)

Training on fold 1/10
Training on fold 2/10
Training on fold 3/10
Training on fold 4/10
Training on fold 5/10
Training on fold 6/10
Training on fold 7/10
Training on fold 8/10
Training on fold 9/10
Training on fold 10/10
0.70000, 0.69379, 0.70356, 0.68855
0.51111, 0.61785, 0.51038, 0.48651
0.60000, 0.60221, 0.60326, 0.60044
0.55556, 0.59660, 0.55830, 0.54991
0.56667, 0.70611, 0.56472, 0.56410
0.63333, 0.64000, 0.63587, 0.62872
0.71111, 0.70352, 0.71492, 0.70431
0.58889, 0.62240, 0.58893, 0.58108
0.61111, 0.62693, 0.61413, 0.60693
0.63333, 0.66906, 0.63933, 0.60045
0.61111, 0.64785, 0.61334, 0.60110
0.05879, 0.03976, 0.06028, 0.06051


In [34]:
# Suguro --------------------------------------------------------------------------------------------
fs = 500
duration = 2
sample = fs * duration
ch = 19
hp = 0.4
lp = 30

data, label = load_target('Sugiyama_JulyData')

data = butterworth_bpf(data, hp, lp, fs)


best_by_ga = [0, 2, 3, 6, 7, 9, 10, 12, 13, 16, 17]
data = data[:,best_by_ga,:]
d,x_test,  l, y_test = train_test_split(data, label, test_size = .15, stratify = label, random_state = 420)
x_train,x_val,  y_train, y_val = train_test_split(d, l, test_size = .10, stratify = l, random_state = 420)

sgr_hist, sgr_acc, sgr_prec, sgr_recall, sgr_f1 = train(x_train, y_train,x_val, y_val, x_test, y_test, 10)
print_eval(sgr_acc, sgr_prec, sgr_recall, sgr_f1)

Training on fold 1/10
Training on fold 2/10
Training on fold 3/10
Training on fold 4/10
Training on fold 5/10
Training on fold 6/10
Training on fold 7/10
Training on fold 8/10
Training on fold 9/10
Training on fold 10/10
0.43333, 0.49071, 0.43676, 0.39276
0.56667, 0.52667, 0.57263, 0.54105
0.66667, 0.67891, 0.66848, 0.65701
0.60000, 0.63014, 0.60227, 0.57703
0.57778, 0.62818, 0.58202, 0.54022
0.63333, 0.71190, 0.63735, 0.61276
0.63333, 0.63228, 0.63735, 0.63083
0.61111, 0.63706, 0.61117, 0.61028
0.67778, 0.67924, 0.68083, 0.67227
0.52222, 0.57992, 0.52717, 0.49126
0.59222, 0.61950, 0.59560, 0.57255
0.06904, 0.06563, 0.06854, 0.08031


In [37]:
# 3 Subjects---------------------------------------------------------------------------------------------
fs = 500
duration = 2
sample = fs * duration
ch = 19
hp = 0.4
lp = 30

data1, label1 = load_target('Takahashi_JulyData')
data2, label2 = load_target('Suguro_JulyData')
data3, label3 = load_target('Sugiyama_JulyData')
data = np.vstack([data1, data2, data3])
label = np.vstack([label1, label2, label3])
data = butterworth_bpf(data, hp, lp, fs)

best_by_ga = [0, 1, 2, 3, 5, 8, 15, 16, 17, 18]
data = data[:,best_by_ga,:]

d,x_test,  l, y_test = train_test_split(data, label, test_size = .15, stratify = label, random_state = 420)
x_train,x_val,  y_train, y_val = train_test_split(d, l, test_size = .10, stratify = l, random_state = 420)

all_hist, all_acc, all_prec, all_recall, all_f1 = train(x_train, y_train,x_val, y_val, x_test, y_test, 25)
print_eval(all_acc, all_prec, all_recall, all_f1)

Training on fold 1/10
Training on fold 2/10
Training on fold 3/10
Training on fold 4/10
Training on fold 5/10
Training on fold 6/10
Training on fold 7/10
Training on fold 8/10
Training on fold 9/10
Training on fold 10/10
0.51111, 0.49357, 0.51246, 0.48381
0.56667, 0.55063, 0.56865, 0.53291
0.54074, 0.53564, 0.54181, 0.53585
0.60370, 0.60139, 0.60568, 0.56880
0.53333, 0.60208, 0.53605, 0.46573
0.42963, 0.56664, 0.43108, 0.38099
0.57037, 0.55721, 0.57205, 0.54643
0.58148, 0.60223, 0.58363, 0.53111
0.59259, 0.58455, 0.59383, 0.58518
0.44074, 0.49276, 0.44304, 0.36706
0.53704, 0.55867, 0.53883, 0.49979
0.05745, 0.03939, 0.05744, 0.07124
